In [ ]:
import torch #파이토치
import torchvision.datasets as dsets #MNIST 데이터셋을 불러옴
import torchvision.transforms as transforms #이미지 변환 툴
import torch.nn.init #텐서에 초기값을 줌

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
trainset = dsets.FashionMNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transforms.ToTensor()
)

testset = dsets.FashionMNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transforms.ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



In [ ]:
len(trainset), len(testset)

(60000, 10000)

## Trial 1

learning_rate = 0.1
training_epochs = 15 
batch_size = 100

drop out = 0.5

batch norm 시행 X

(1, 32, 64, 128, *625, *10)

>> 0.1

In [ ]:
learning_rate = 0.1
training_epochs = 15 
batch_size = 100

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64, kernel_size = 3, stride =1, padding =1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size = 3, stride =1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
    
    self.fc1 = torch.nn.Linear(4*4*128, 625, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
    
    self.fc2 = torch.nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out



In [ ]:
model = CNN().to(device)

# 비용함수(소프트맥스 함수 포함)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# 총 배치 갯수
total_batch = len(train_loader)
print(total_batch)

600


In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()


    avg_cost += cost / total_batch
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 2.31052876
[Epoch:    2] cost = 2.30992913
[Epoch:    3] cost = 2.30939889
[Epoch:    4] cost = 2.30975866
[Epoch:    5] cost = 2.31010485
[Epoch:    6] cost = 2.30968714
[Epoch:    7] cost = 2.30918145
[Epoch:    8] cost = 2.30981493
[Epoch:    9] cost = 2.30903506
[Epoch:   10] cost = 2.30918717
[Epoch:   11] cost = 2.30986905
[Epoch:   12] cost = 2.30990911
[Epoch:   13] cost = 2.30972052
[Epoch:   14] cost = 2.31013823
[Epoch:   15] cost = 2.30962086


In [ ]:
correct = 0

with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    pred = output.max(1, keepdim = True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()

  test_accuracy = correct / len(test_loader.dataset)
  print("accuracy:", test_accuracy)

accuracy: 0.1


## Trial 2

learning_rate = 0.01
training_epochs = 100 
batch_size = 100

drop out = 0.5

batch norm 시행

(1, 32, 64, 128, *625, *10)

>> 0.9068

In [ ]:
learning_rate = 0.01
training_epochs = 100 
batch_size = 100

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.BatchNorm2d(32),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64, kernel_size = 3, stride =1, padding =1),
        torch.nn.BatchNorm2d(64),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size = 3, stride =1, padding = 1),
        torch.nn.BatchNorm2d(128),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
    
    self.fc1 = torch.nn.Linear(4*4*128, 625, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
    
    self.fc2 = torch.nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out



In [ ]:
model = CNN().to(device)

# 비용함수(소프트맥스 함수 포함)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# 총 배치 갯수
total_batch = len(train_loader)
print(total_batch)

600


In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()


    avg_cost += cost / total_batch
  if epoch % 10 ==0:
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 1.1916815
[Epoch:   11] cost = 0.2388511
[Epoch:   21] cost = 0.188518345
[Epoch:   31] cost = 0.154398516
[Epoch:   41] cost = 0.14179194
[Epoch:   51] cost = 0.127223432
[Epoch:   61] cost = 0.115531892
[Epoch:   71] cost = 0.115740567
[Epoch:   81] cost = 0.102586001
[Epoch:   91] cost = 0.0960200951


In [ ]:
correct = 0

with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    pred = output.max(1, keepdim = True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()

  test_accuracy = correct / len(test_loader.dataset)
  print("accuracy:", test_accuracy)

accuracy: 0.9068


## Trial 3

learning_rate = 0.01
training_epochs = 100 
batch_size = 100

drop out 2d = 0.5 ??

batch norm 시행

(1, 32, 64, 128, *625, *10)

>> 0.9067

In [ ]:
learning_rate = 0.01
training_epochs = 100 
batch_size = 100

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.BatchNorm2d(32),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64, kernel_size = 3, stride =1, padding =1),
        torch.nn.BatchNorm2d(64),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size = 3, stride =1, padding = 1),
        torch.nn.BatchNorm2d(128),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
    
    self.fc1 = torch.nn.Linear(4*4*128, 625, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
    
    self.fc2 = torch.nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out



In [ ]:
model = CNN().to(device)

# 비용함수(소프트맥스 함수 포함)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()


    avg_cost += cost / total_batch
  if epoch % 10 ==0:
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 1.00172722
[Epoch:   11] cost = 0.227652192
[Epoch:   21] cost = 0.172279388
[Epoch:   31] cost = 0.151078358
[Epoch:   41] cost = 0.131218001
[Epoch:   51] cost = 0.125896782
[Epoch:   61] cost = 0.121119246
[Epoch:   71] cost = 0.108819827
[Epoch:   81] cost = 0.0839184299
[Epoch:   91] cost = 0.101883814


In [ ]:
correct = 0

with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    pred = output.max(1, keepdim = True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()

  test_accuracy = correct / len(test_loader.dataset)
  print("accuracy:", test_accuracy)

accuracy: 0.9067


## Trial 4

learning_rate = 0.01
training_epochs = 100 
batch_size = 100

drop out 2d = 0.25 ??

batch norm 시행

(1, 32, 64, 128, *625, *10)

>> 0.8899



In [ ]:
learning_rate = 0.01
training_epochs = 100 
batch_size = 100

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.25

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.BatchNorm2d(32),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64, kernel_size = 3, stride =1, padding =1),
        torch.nn.BatchNorm2d(64),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size = 3, stride =1, padding = 1),
        torch.nn.BatchNorm2d(128),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
    
    self.fc1 = torch.nn.Linear(4*4*128, 625, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
    
    self.fc2 = torch.nn.Linear(625, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out



In [ ]:
model = CNN().to(device)

# 비용함수(소프트맥스 함수 포함)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()


    avg_cost += cost / total_batch
  if epoch % 10 ==0:
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 1.29671502
[Epoch:   11] cost = 0.291220933
[Epoch:   21] cost = 0.259051949
[Epoch:   31] cost = 0.220814347
[Epoch:   41] cost = 0.2062013
[Epoch:   51] cost = 0.192415148
[Epoch:   61] cost = 0.185972169
[Epoch:   71] cost = 0.176923364
[Epoch:   81] cost = 0.192162454
[Epoch:   91] cost = 0.178549111


In [ ]:
correct = 0

with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    pred = output.max(1, keepdim = True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()

  test_accuracy = correct / len(test_loader.dataset)
  print("accuracy:", test_accuracy)

accuracy: 0.8899


## Trial 5

learning_rate = 0.01
training_epochs = 100 
batch_size = 100

drop out 2d = 0.5

batch norm 시행

(1, 32, 64, *600, *120, *10)

>> 0.8614



In [ ]:
learning_rate = 0.01
training_epochs = 100 
batch_size = 100

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.BatchNorm2d(32),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64, kernel_size = 3, stride =1),
        torch.nn.BatchNorm2d(64),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    self.fc1 = torch.nn.Linear(6*6*64, 600, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer3 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
    
    self.fc2 = torch.nn.Linear(600, 120, bias = True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc2,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
    
    self.fc3 = torch.nn.Linear(120, 10, bias = True)
    torch.nn.init.xavier_uniform_(self.fc3.weight)


  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.fc3(out)
    return out



In [ ]:
model = CNN().to(device)

# 비용함수(소프트맥스 함수 포함)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in train_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()


    avg_cost += cost / total_batch
  if epoch % 10 ==0:
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 1.58302248
[Epoch:   11] cost = 0.372135371
[Epoch:   21] cost = 0.355362862
[Epoch:   31] cost = 0.338968575
[Epoch:   41] cost = 0.341228068
[Epoch:   51] cost = 0.336336851
[Epoch:   61] cost = 0.337168485
[Epoch:   71] cost = 0.363218397
[Epoch:   81] cost = 0.34629035
[Epoch:   91] cost = 0.36478588


In [ ]:
correct = 0

with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    pred = output.max(1, keepdim = True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()

  test_accuracy = correct / len(test_loader.dataset)
  print("accuracy:", test_accuracy)

accuracy: 0.8614
